In [ ]:
# importing necessary libraries
# Chalo bhai, sab zaroori libraries import kar lete hain. Koi bhoolna mat, warna 'ModuleNotFoundError' ka error aayega aur poori raat debugging karni padegi! 🤪
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
# Ab apni CSV file upload karte hain. Ye step mat bhoolna, warna 'FileNotFoundError' aayega aur sir dukh jayega! 😅
from google.colab import files
uploaded = files.upload()


In [ ]:
# -------------------------------
# STEP 1: DATA LOAD AUR PREPROCESSING
# -------------------------------

# Dataset ko load karte hain. Ek baar check kar lena file name sahi hai ya nahi. Spelling mistake se project latak jata hai. 😉
df = pd.read_csv("irrigation_machine.csv")


In [ ]:
# Pehle 5 rows dekhte hain. Jisse data ki shakal pata chal jaye. 'df.tail()' se last 5 rows bhi dekh sakte hain, just for fun. 😄
df.head()

In [ ]:
# Ab data ki kundli nikalte hain. Pura info, jaise kitne columns, koi missing values toh nahi... agar missing hai toh agle step mein dhyan rakhna padega. 🧐
df.info()

In [ ]:
# 'Unnamed: 0' column ek फालतू column lag raha hai. Ise nikal dete hain. Bheed kam, kaam zyada. 💪
df = df.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
df.describe() # Data ka stats, jaise min, max, average. Isse outliers bhi pakad sakte hain. 🕵️‍♂️

In [ ]:
# -------------------------------
# STEP 2: FEATURES AUR LABELS
# -------------------------------

# Input (X) aur Output (y) ko alag-alag karte hain. X hamare sensors hain, aur y hamari sprinkler's. Simple, right? 🧠
X = df.iloc[:, 0:20]   # sensor_0 to sensor_19

y = df.iloc[:, 20:]    # parcel_0, parcel_1, parcel_2

In [ ]:
X.sample(10) # 10 random samples of X, bas verify karne ke liye ki sab theek hai. 👌

In [ ]:
y.sample(10) # Aur yahan 10 random samples of y. Check karlo ki 0 aur 1 hi hai na. Binary classification hai bhai! 🤖

In [ ]:
X.info() # X ka info. Ab saare columns float64 hone chahiye. Jo nahi hain, unko sahi karo! 😠

In [ ]:
y.info() # y ka info. Saare columns int64 hain, perfect! Koi galti nahi. ✅

In [ ]:
X # Poora X print kar rahe hain. Screen bhar jayegi, lekin satisfaction milega. 😌

In [ ]:
X.shape, y.shape # Shapes dekh rahe hain. Dono mein rows same honi chahiye. Warna 'ValueError' aayega. 😟

In [ ]:
scaler = MinMaxScaler()
# Ab data ko scale kar rahe hain. Jisse saare features 0 se 1 ke beech mein aa jayein. Warna model partiality karega. 😒
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
# -------------------------------
# STEP 3: TRAIN-TEST SPLIT
# -------------------------------

# Ab data ko training aur testing ke liye split karte hain. 80% training, 20% testing. Random_state se har baar same split milega. 🤝
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape # Fir se shape check! Dil ko tasalli milti hai. ❤️

In [ ]:
# -------------------------------
# STEP 4: MODEL TRAINING
# -------------------------------

# Apne model ko banate hain. RandomForestClassifier is powerful, but MultiOutputClassifier is the real boss here. 😎
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

# Humne custom hyperparameters set kiye hain. Isse model ki performance aur badhegi.
rf = RandomForestClassifier(
    n_estimators=200,         # Kitne trees honge? Zyada trees, zyada accuracy, lekin training time bhi zyada. 🌲
    max_depth=10,             # Har tree kitna deep jayega? Isse overfitting avoid hota hai. 🌳
    min_samples_split=4,      # Node ko split karne ke liye kitne samples chahiye? Important hai! 🧐
    min_samples_leaf=2,       # Har leaf mein kitne samples honge? Overfitting se bachata hai. 🍃
    max_features='sqrt',      # Har split pe kitne features dekhega? 'sqrt' generally accha kaam karta hai. ✨
    random_state=42
)

# MultiOutputClassifier wrap karte hain, kyuki hume ek sath 3 cheeze predict karni hai. 🤝
model = MultiOutputClassifier(rf)

# Model ko training data pe fit karte hain. Ab shuru hota hai asli khel. 🚀
model.fit(X_train, y_train)


In [ ]:
# -------------------------------
# STEP 5: MODEL EVALUATION
# -------------------------------

# Model ne testing data pe kya predict kiya? Chalo dekhte hain. 🤞
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=y.columns)) # Report card aa gaya! Precision, recall, F1-score check karo. 📊

In [ ]:
print(df[['parcel_0', 'parcel_1', 'parcel_2']].sum()) # Original data mein total kitni baar sprinklers ON the? Tally kar lo. 🧐

In [ ]:
import matplotlib.pyplot as plt

# Ab data ko plot karte hain. Charts se sab kuch clear ho jata hai. 📈
conditions = {
    "Parcel 0 ON": df['parcel_0'],
    "Parcel 1 ON": df['parcel_1'],
    "Parcel 2 ON": df['parcel_2'],
    "Parcel 0 & 1 ON": df['parcel_0'] & df['parcel_1'],
    "Parcel 0 & 2 ON": df['parcel_0'] & df['parcel_2'],
    "Parcel 1 & 2 ON": df['parcel_1'] & df['parcel_2'],
    "All Parcels ON": df['parcel_0'] & df['parcel_1'] & df['parcel_2'],
}

# Vertically stacked subplots banate hain. Har condition ke liye alag plot. 🖼️
fig, axs = plt.subplots(nrows=len(conditions), figsize=(10,15), sharex=True)

# Loop chala ke har condition ka plot banate hain. Ye magic hai! ✨
for ax, (title, condition) in zip(axs, conditions.items()):
    ax.step(df.index, condition.astype(int), where='post', linewidth=1, color='teal')
    ax.set_title(f"Sprinkler - {title}")
    ax.set_ylabel("Status")
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['OFF', 'ON'])
   

# Last subplot mein x-axis label. Sahi plotting ka rule hai ye. 📌
axs[-1].set_xlabel("Time Index (Row Number)")

# Plot dikha do! 👀
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Ab ek aur plot. Individual pumps aur combined coverage. Ek hi frame mein saari kahani. 🎬
any_pump_on = (df['parcel_0'] == 1) | (df['parcel_1'] == 1) | (df['parcel_2'] == 1)

plt.figure(figsize=(15, 5))

# Har pump ka status plot karte hain. Blue, Orange, Green - sab clear hai. 🎨
plt.step(df.index, df['parcel_0'], where='post', linewidth=2, label='Parcel 0 Pump', color='blue')
plt.step(df.index, df['parcel_1'], where='post', linewidth=2, label='Parcel 1 Pump', color='orange')
plt.step(df.index, df['parcel_2'], where='post', linewidth=2, label='Parcel 2 Pump', color='green')

plt.title("Pump Activity and Combined Farm Coverage")
plt.xlabel("Time Index (Row Number)")
plt.ylabel("Status")
plt.yticks([0, 1], ['OFF', 'ON'])
plt.legend(loc='upper right')
plt.show()


In [ ]:
import joblib
from sklearn.pipeline import Pipeline

# Sabse important step: model ko save kar rahe hain. Ye hamare project ka final output hai. 📦
joblib.dump(model, "Farm_Irrigation_System.pkl")
print("Model saved as 'Farm_Irrigation_System.pkl'. Ab isse Streamlit app mein use karenge. 🎉")